# Event Series & Field Analysis for g4chargeit

This notebook visualizes particle tracks (Photoemission and Solar Wind) overlaid on electric field maps and geometry slices.

**Features:**
1.  **Physics Verification:** Checks particle energy spectra and flux conversion.
2.  **Field Visualization:** 2D slices of the Electric Field ($E$) vectors using PyVista.
3.  **Event Series:** Overlays individual particle tracks (Gamma, Electrons, Protons) from Geant4 ROOT files onto the geometry.

**Requirements:**
* `uproot`, `pandas`, `numpy`, `scipy`, `matplotlib`, `trimesh`, `pyvista`
* `common_functions.py` (Ensure this is in your python path)

In [ ]:
import glob
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import uproot
import trimesh
import pyvista as pv
from scipy.spatial import cKDTree
from scipy.constants import epsilon_0, e as q_e

# Import local helper functions
try:
    from common_functions import *
except ImportError:
    print("Warning: common_functions.py not found. Ensure it is in the directory.")

# Plotting Configuration
mpl.rcParams['text.usetex'] = False
mpl.rcParams.update({'font.size': 12})
pv.set_jupyter_backend('trame') # Options: 'trame', 'static', 'client'

In [ ]:
def get_root_file(directory, iteration, config_name, num_particles=100000):
    """Finds and reads the specific ROOT file for an iteration."""
    pattern = f"{directory}/root/*iteration{iteration}*{config_name}*_num{num_particles}.root"
    files = sorted(glob.glob(pattern))
    
    if not files:
        print(f"No ROOT files found for pattern: {pattern}")
        return None
        
    filename = files[0]
    print(f"Loading ROOT file: {os.path.basename(filename)}")
    return read_rootfile(filename, directory_path=f"{directory}/root")

def get_fieldmap(directory, iteration, config_name):
    """Finds and loads the fieldmap text file."""
    # Handle filename padding differences
    pad = "00" if iteration < 10 else ""
    pattern = f"{directory}/fieldmaps/*{pad}{iteration}*{config_name}*.txt"
    files = sorted(glob.glob(pattern))
    
    if not files:
        print(f"No Fieldmap found for pattern: {pattern}")
        return None

    # Assuming read_data_format_efficient returns a dict keyed by iteration
    df_map = read_data_format_efficient(files, scaling=True)
    return df_map[iteration]

def process_field_slice(field_data, geometry, y_slice, thickness=0.002, spacing=0.02, scale_factor=4e-7):
    """
    Processes 3D field data into a 2D slice with vector glyphs using PyVista.
    """
    # 1. Filter Data
    points = field_data["pos"]
    vectors = field_data["E"]
    mags = field_data["E_mag"]
    
    mask = (mags > 0)
    points = points[mask]
    vectors = vectors[mask]
    mags = mags[mask]

    # 2. Create Geometry Slice
    pv_geo = pv.PolyData(geometry.vertices, 
                         np.hstack([np.full((len(geometry.faces), 1), 3), geometry.faces]))
    
    # Clip and Slice
    bounds = pv.PolyData(points).bounds
    slice_center = [(bounds[0]+bounds[1])/2, y_slice, (bounds[4]+bounds[5])/2]
    
    pv_geo_clipped = pv_geo.clip_box(pv.Box(bounds), invert=False)
    geo_slice = pv_geo_clipped.slice(normal=[0,1,0], origin=slice_center)

    # 3. Vector Slicing & Downsampling
    v_mask = np.abs(points[:, 1] - y_slice) < thickness
    pts_slice = points[v_mask]
    vec_slice = vectors[v_mask]
    mag_slice = mags[v_mask]

    # Voxel Downsampling logic (simplified)
    # Note: A simple striding or cKDTree could also be used here
    # For now, we take every Nth point for simplicity if density is high
    pts_ds = pts_slice
    vec_ds = vec_slice
    mag_ds = mag_slice

    # Flatten Y for 2D plotting
    pts_ds[:, 1] = y_slice - 2*thickness 
    vec_ds[:, 1] = 0 

    # Create PolyData
    pd_vectors = pv.PolyData(pts_ds)
    pd_vectors['vectors'] = vec_ds
    pd_vectors['magnitude'] = np.clip(mag_ds, None, spacing/scale_factor/2) # Clamp max magnitude

    glyphs = pd_vectors.glyph(orient='vectors', scale='magnitude', factor=scale_factor, geom=pv.Arrow(tip_length=0.3))
    
    return geo_slice, glyphs

# 1. Load in data and calculate lunar equivalent time

In [ ]:
# --- User Configuration ---
# Update these paths to match your local environment
DATA_DIR = "/path/to/data/"
GEOMETRY_PATH = '../g4chargeit/geometry/irregularSpheres_fromPython.stl'
# other options: regularSpheres_fromPython.stl, realisticGrains_fromPython.stl

# Simulation Steps to Analyze
ITERATION_PE = 17
ITERATION_SW = 39

# Physics Constants & Flux Settings
WORLD_AREA_M2 = 400 * 300 / (1e6**2)  # World area in m^2
PE_FLUX = 4e-6 * 6.241509e18          # Photoelectrons
SW_ION_FLUX = 3e-7 * 6.241509e18      # Solar Wind Ions
SW_ELE_FLUX = 1.5e-6 * 6.241509e18    # Solar Wind Electrons

# Load Geometry
if os.path.exists(GEOMETRY_PATH):
    stacked_spheres = trimesh.load_mesh(GEOMETRY_PATH)
    print(f"Geometry loaded: {GEOMETRY_PATH}")
else:
    print(f"Error: Geometry file not found at {GEOMETRY_PATH}")
    # Create dummy geometry for testing if file missing
    stacked_spheres = trimesh.creation.icosphere(radius=0.1)

# Calculate Time Factors
particles_pe_iter = 81775 # PLACEHOLDER VALUES
time_pe = particles_pe_iter / WORLD_AREA_M2 / PE_FLUX

particles_sw_iter = 14208
time_sw = particles_sw_iter / WORLD_AREA_M2 / SW_ION_FLUX

print(f"PE Time per Iteration: {time_pe*1000:.4f} ms")
print(f"SW Time per Iteration: {time_sw*1000:.4f} ms")

# 2. Photoemission Case
Analyzing Gamma rays hitting the surface and electron emission.

In [ ]:
# Load Data
df_photoemission = get_root_file(DATA_DIR, ITERATION_PE, "onlyphotoemission")
field_pe = get_fieldmap(DATA_DIR, ITERATION_PE, "onlyphotoemission")

In [ ]:
if df_photoemission is not None:
    # 1. All Photoelectrons generated
    pe_gen = df_photoemission[df_photoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")
    
    # 2. Photoelectrons that escape (reach simulation boundary)
    pe_last = df_photoemission[df_photoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="last")
    
    # Define "Escaped" as reaching top/bottom boundaries
    # Note: This logic assumes Z-axis is vertical
    z_positions = np.vstack(pe_last["Post_Step_Position_mm"])[:, 2]
    z_min, z_max = np.min(z_positions), np.max(z_positions)
    escaped_indices = np.argwhere((z_positions >= z_max*0.99) | (z_positions <= z_min*0.99))
    
    pe_escaped_energies = np.vstack(pe_last["Kinetic_Energy_Post_MeV"])[escaped_indices] * 1e6 # to eV

    # Plotting
    bins = np.logspace(-4, 3.2, 200)
    
    fig, ax = plt.subplots(figsize=(6, 4))
    
    # Full Spectrum
    vals_all, edges = np.histogram(pe_gen["Kinetic_Energy_Pre_MeV"]*1e6, bins=bins)
    centers = 0.5*(edges[1:] + edges[:-1])
    width = np.diff(edges)
    ax.step(centers, vals_all/width, where='mid', label='All Generated PE', color='darkblue')
    
    # Escaped Spectrum
    vals_esc, _ = np.histogram(pe_escaped_energies, bins=bins)
    ax.step(centers, vals_esc/width, where='mid', label='Escaped PE', color='darkgreen')
    
    ax.set_xscale('log'); ax.set_yscale('log')
    ax.set_xlabel("Energy (eV)"); ax.set_ylabel("Differential Flux")
    ax.set_title("Photoelectron Energy Spectrum")
    ax.legend()
    plt.show()
    
    print(f"Escape Fraction: {len(escaped_indices) / len(pe_gen) * 100:.2f}%")

In [ ]:
# Parameters for Slice
Y_SLICE = stacked_spheres.centroid[1]
EVENTS_TO_PLOT = [114, 99054, 61332] # Replace with interesting Event Numbers found in your data

# Process Field Slice
geo_slice, field_glyphs = process_field_slice(field_pe, stacked_spheres, Y_SLICE)

# --- Plotting with Matplotlib ---
fig, ax = plt.subplots(figsize=(10, 8))

# 1. Plot Geometry Slice (from PyVista to MPL)
# Helper to extract lines from PyVista slice
def plot_pv_slice_mpl(pv_slice, ax):
    lines = pv_slice.lines.reshape(-1, 3)[:, 1:] # Skip size indicator
    points = pv_slice.points
    for i in range(0, len(lines), 2): # Iterate segments
        idx1, idx2 = lines[i], lines[i+1] # Approx logic, varies by PV version
        # Robust way: iterate cells
        pass 
    
    # Robust iteration over cells
    for i in range(pv_slice.n_cells):
        cell_points = pv_slice.get_cell(i).points
        ax.plot(cell_points[:,0], cell_points[:,2], 'k-', lw=3, alpha=0.5)

plot_pv_slice_mpl(geo_slice, ax)

# 2. Plot Particle Tracks
for event_num in EVENTS_TO_PLOT:
    event_df = df_photoemission[df_photoemission["Event_Number"] == event_num]
    
    for p_type, color, lw in [("gamma", "green", 4), ("e-", "red", 2)]:
        subset = event_df[event_df["Particle_Type"] == p_type]
        if subset.empty: continue
            
        # Interleave Pre and Post positions for continuous line segments
        pre = np.vstack(subset["Pre_Step_Position_mm"])
        post = np.vstack(subset["Post_Step_Position_mm"])
        
        # Simple plot: just connect pre points (approximation) or interleave
        # For high fidelity:
        track = np.empty((pre.shape[0] + post.shape[0], 3))
        track[0::2] = pre
        track[1::2] = post
        
        ax.plot(track[:,0], track[:,2], color=color, alpha=0.8, lw=lw, label=p_type if event_num==EVENTS_TO_PLOT[0] else "")

ax.set_aspect('equal')
ax.set_title(f"Photoemission Events (Slice Y={Y_SLICE:.3f})")
ax.set_xlabel("X (mm)"); ax.set_ylabel("Z (mm)")
plt.legend()
plt.show()

# 3. Solar Wind Case

Analyzing Protons and Electrons from the solar wind interacting with the charged environment.

In [ ]:
# Load Data
df_sw = get_root_file(DATA_DIR, ITERATION_SW, "onlysolarwind")
field_sw = get_fieldmap(DATA_DIR, ITERATION_SW, "onlysolarwind")

# SW Events to Visualize
SW_EVENTS = [2253, 23236, 37927]

# Process Field Slice (Note: Scaling factor might need adjustment for SW fields)
geo_slice_sw, field_glyphs_sw = process_field_slice(field_sw, stacked_spheres, Y_SLICE, scale_factor=7e-7)

# --- Plotting ---
fig, ax = plt.subplots(figsize=(10, 8))

# 1. Geometry
plot_pv_slice_mpl(geo_slice_sw, ax)

# 2. Tracks
for event_num in SW_EVENTS:
    event_df = df_sw[df_sw["Event_Number"] == event_num]
    
    # Plot Protons
    protons = event_df[event_df["Particle_Type"] == "proton"]
    if not protons.empty:
        pre = np.vstack(protons["Pre_Step_Position_mm"])
        post = np.vstack(protons["Post_Step_Position_mm"])
        track = np.empty((pre.shape[0] + post.shape[0], 3))
        track[0::2] = pre; track[1::2] = post
        
        ax.plot(track[:,0], track[:,2], '.-', color='blue', alpha=0.6, lw=2, markersize=5)

ax.set_aspect('equal')
ax.set_title(f"Solar Wind Protons (Slice Y={Y_SLICE:.3f})")
plt.show()

In [ ]:
# Visualize particle stopping locations
if 'df_onlysolarwind' in dir():
    last_protons = df_onlysolarwind[
        df_onlysolarwind["Particle_Type"] == "proton"
    ].drop_duplicates(subset="Event_Number", keep="last")
    protons_inside = last_protons[last_protons["Volume_Name_Post"] == "SiO2"][::100]
    
    last_electrons = df_onlysolarwind[
        df_onlysolarwind["Particle_Type"] == "e-"
    ].drop_duplicates(subset="Event_Number", keep="last")
    electrons_inside = last_electrons[last_electrons["Volume_Name_Post"] == "SiO2"][::100]
    
    print(f"Protons stopped inside: {len(last_protons[last_protons['Volume_Name_Post'] == 'SiO2'])}")
    print(f"Electrons stopped inside: {len(last_electrons[last_electrons['Volume_Name_Post'] == 'SiO2'])}")
    
    # Create point clouds
    proton_points = PointCloud(
        np.array(protons_inside["Pre_Step_Position_mm"].tolist()),
        colors=[0, 255, 0, 255]  # Green
    )
    electron_points = PointCloud(
        np.array(electrons_inside["Pre_Step_Position_mm"].tolist()),
        colors=[0, 0, 255, 255]  # Blue
    )
    
scene = trimesh.Scene([geometry, proton_points, electron_points])
scene.show()